In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np
import unicodedata
import re
import random
import argparse
import pickle
import logging

logger = logging.getLogger()
 
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
 
# Setup file handler
fhandler  = logging.FileHandler('my.log')
fhandler.setLevel(logging.DEBUG)
fhandler.setFormatter(formatter)
 
# Configure stream handler for the cells
chandler = logging.StreamHandler()
chandler.setLevel(logging.DEBUG)
chandler.setFormatter(formatter)
 
# Add both handlers
logger.addHandler(fhandler)
logger.addHandler(chandler)
logger.setLevel(logging.DEBUG)
 
# Show the handlers
logger.handlers


titanic_file = '/home/wrench/下载/ML_Tech_Stack/Natural_language_process/myonvif_info.csv'
test_file = '/home/wrench/下载/ML_Tech_Stack/Natural_language_process/myonvif_test.csv'
#titanic_lines = tf.data.TextLineDataset(titanic_file)
#for line in titanic_lines.take(10):
#      print(line.numpy())
#np.set_printoptions(precision=3, suppress=True) 

#LABEL_COLUMN = 'brand'
#LABELS = [0, 1]
#def get_dataset(file_path):
#    dataset = tf.data.experimental.make_csv_dataset(
#      titanic_file,
#      batch_size=1, # 为了示例更容易展示，手动设置较小的值
#      label_name=LABEL_COLUMN,
#      na_value="?",
#      num_epochs=1,
#      ignore_errors=True)
#    return dataset       
#        
#raw_train_data = get_dataset(titanic_file)
#raw_test_data = get_dataset(test_file)
#examples, labels = next(iter(raw_train_data)) # 第一个批次
#print("EXAMPLES: \n", examples, "\n")
#print("LABELS: \n", labels)

test_partition = 0.1
BATCH_SIZE = 2
BUFFER_SIZE = 10000
MAX_LENGTH = 50

In [2]:
def pack_features_vector(features):
    """Pack the features into a single array."""
    features = tf.stack(list(features.values()), axis=1)
    return features

In [3]:
def unicode_to_ascii(s):
    return "".join(
        c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c) != "Mn"
    )

def preprocess_sentence(w):

    w = unicode_to_ascii(w.lower().strip())

    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", w)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    string = re.sub(r"www", "", string)
    string = re.sub(r"onvif", "", string)
    string = re.sub(r"org", "", string)
    return string.strip().lower()

In [4]:
#class Dataset:
#    def __init__(
#        self,
#        filename="",
#        vocab_dim=10000,
#        max_length=2100,
#        buffer_size=20000,
#        batch_size=64,
#        num_classes=5,
#        classes=[],
#    ):
#        self.input_filename = filename
#        self.classes_filename = filename + ".classes"
#        self.train_filename = filename + ".train"
#        self.test_filename = filename + ".test"
#        self.vocabulary_size = vocab_dim
#        self.max_length = max_length
#        self.batch_size = batch_size
#        self.buffer_size = buffer_size
#        self.num_classes = num_classes
#        self.classes = classes
#
#    def build_train_test(self, test=0.2):
#        """ This function takes the input file and splits into training and testing """
#
#        labels = {}
#
#        file_lines = []
#        for ix, i in enumerate(open(self.input_filename)):
#            i = i.strip().split("\t")
#            labels[i[1]] = True
#            file_lines.append(ix)
#
#        random.shuffle(file_lines)
#        test_limit = int(len(file_lines) * test)
#
#        test_dict = {i: True for i in file_lines[0:test_limit]}
#
#        fo_test = open(self.test_filename, "w")
#        fo_train = open(self.train_filename, "w")
#        for ix, i in enumerate(open(self.input_filename)):
#            try:
#                assert test_dict[ix]
#                fo_test.write(i)
#            except:
#                fo_train.write(i)
#
#        fo = open(self.classes_filename, "w")
#        for i in labels:
#            fo.write(i + "\n")
#        fo.close()
#
#    def format_train_test(self):
#        filenames = [self.train_filename]
#        train_dataset_tf = tf.data.Dataset.from_tensor_slices(filenames)
#        train_dataset_tf = train_dataset_tf.flat_map(
#            lambda filename: (tf.data.TextLineDataset(filename))
#        )
#
#        filenames = [self.test_filename]
#        test_dataset_tf = tf.data.Dataset.from_tensor_slices(filenames)
#        test_dataset_tf = test_dataset_tf.flat_map(
#            lambda filename: (tf.data.TextLineDataset(filename))
#        )
#
#        return train_dataset_tf, test_dataset_tf
#
#    def format_dataset(self):
#        filenames = [self.train_filename, self.test_filename]
#        test_dataset_tf = tf.data.Dataset.from_tensor_slices(filenames)
#        test_dataset_tf = test_dataset_tf.flat_map(
#            lambda filename: (tf.data.TextLineDataset(filename))
#        )
#        return test_dataset_tf
#
#    def tokenizer(self, train_examples,Classlist):
##        for examples , labels in  raw_train_data:
##             print (examples['context'].numpy())
#        self.tokenizer_source = tfds.features.text.SubwordTextEncoder.build_from_corpus( 
#        (b"".join(examples['context'].numpy().tolist()) for examples , labels in  raw_train_data),
#        target_vocab_size=2**13,
#        )
#        self.tokenizer_target = tfds.features.ClassLabel(names=Classlist)
#        return self.tokenizer_source, self.tokenizer_target
#
#    def encode(self, string_Tensor,k_Tensor):
##        print('source')
##        print(string_Tensor)
##        print('target')
##        print(k_Tensor)
#        source = (
#            [self.tokenizer_source.vocab_size]
#            + self.tokenizer_source.encode(
#                preprocess_sentence(
#                    b"".join(string_Tensor.numpy().tolist()).decode("UTF-8")
#                ).encode()
#            )
#            + [self.tokenizer_source.vocab_size + 1]
#        )
#
#        target = self.tokenizer_target.encode_example(
#            b"".join(k_Tensor.numpy().tolist()).decode("UTF-8")
#        )
#        #这里在下方代码for one_element in train_dataset:  会执行  ，随便查看数据
##        print('source')
##        print(string_Tensor)
##        print('target')
##        print(target)
#        return source, target
#
#    def tf_encode(self, string_Tensor,k_Tensor):
#        return tf.py_function(self.encode, [string_Tensor,k_Tensor], [tf.int64, tf.int64])
#
#    def filter_max_length(self, x, y):
#        return tf.logical_and(
#            tf.size(x) <= self.max_length, tf.size(x) <= self.max_length
#        )

class Dataset:
    def __init__(
        self,
        filename="",
        vocab_dim=10000,
        max_length=40,
        buffer_size=20000,
        batch_size=64,
        num_classes=5,
        classes=[],
    ):
        self.input_filename = filename
        self.classes_filename = filename + ".classes"
        self.train_filename = filename + ".train"
        self.test_filename = filename + ".test"
        self.vocabulary_size = vocab_dim
        self.max_length = max_length
        self.batch_size = batch_size
        self.buffer_size = buffer_size
        self.num_classes = num_classes
        self.classes = classes
        self.labels_list=['Dahua','Hikvision']

    def build_train_test(self, test=0.2):
        """ This function takes the input file and splits into training and testing """

        labels = {}

        file_lines = []
        for ix, i in enumerate(open(self.input_filename)):
            i = i.strip().split("\t")
            labels[i[1]] = True
            file_lines.append(ix)

        random.shuffle(file_lines)
        test_limit = int(len(file_lines) * test)

        test_dict = {i: True for i in file_lines[0:test_limit]}

        fo_test = open(self.test_filename, "w")
        fo_train = open(self.train_filename, "w")
        for ix, i in enumerate(open(self.input_filename)):
            try:
                assert test_dict[ix]
                fo_test.write(i)
            except:
                fo_train.write(i)

        fo = open(self.classes_filename, "w")
        for i in labels:
            fo.write(i + "\n")
        fo.close()

    def format_train_test(self):
        filenames = [self.train_filename]
        train_dataset_tf = tf.data.Dataset.from_tensor_slices(filenames)
        train_dataset_tf = train_dataset_tf.flat_map(
            lambda filename: (tf.data.TextLineDataset(filename))
        )

        filenames = [self.test_filename]
        test_dataset_tf = tf.data.Dataset.from_tensor_slices(filenames)
        test_dataset_tf = test_dataset_tf.flat_map(
            lambda filename: (tf.data.TextLineDataset(filename))
        )

        return train_dataset_tf, test_dataset_tf

    def format_dataset(self):
        filenames = [self.train_filename, self.test_filename]
        test_dataset_tf = tf.data.Dataset.from_tensor_slices(filenames)
        test_dataset_tf = test_dataset_tf.flat_map(
            lambda filename: (tf.data.TextLineDataset(filename))
        )
        return test_dataset_tf

    def tokenizer(self, train_examples):
        self.tokenizer_source = tfds.features.text.SubwordTextEncoder.build_from_corpus(
            (
                preprocess_sentence(pt.numpy().decode(
                    "UTF-8").split("\t")[0]).encode()
                for pt in train_examples
            ),
            target_vocab_size=self.vocabulary_size,
        )

        self.tokenizer_target = tfds.features.ClassLabel(
            names_file=self.classes_filename
        )

        return self.tokenizer_source, self.tokenizer_target

    def encode(self, string):
        source = (
            [self.tokenizer_source.vocab_size]
            + self.tokenizer_source.encode(
                preprocess_sentence(
                    string.numpy().decode("UTF-8").split("\t")[0]
                ).encode()
            )
            + [self.tokenizer_source.vocab_size + 1]
        )

        target = self.tokenizer_target.encode_example(
            string.numpy().decode("UTF-8").split("\t")[1]
        )

        return source, target

    def tf_encode(self, string):
        return tf.py_function(self.encode, [string], [tf.int64, tf.int64])

    def filter_max_length(self, x, y):
        return tf.logical_and(
            tf.size(x) <= self.max_length, tf.size(x) <= self.max_length
        )

In [5]:
#examples_list = []
#
#
#labels_list=['Dahua','Hikvision']
##print(examples)
#dataset = Dataset()
#
#tokenizer_source,tokenizer_target = dataset.tokenizer(raw_train_data ,labels_list)
##
##
##    
##print("dasdsadas")
#db_train = tf.data.Dataset.from_tensor_slices(
#    (
#        [examples['context'] for examples ,labels in raw_train_data ],
#        [labels for examples ,labels in raw_train_data ]
#    )
#)
#
#db_test = tf.data.Dataset.from_tensor_slices(
#    (
#        [examples['context'] for examples ,labels in raw_test_data ],
#        [labels for examples ,labels in raw_test_data ]
#    )
#)
#
#
#
##for (step_idx, (inp, tar)) in enumerate(db_train):
##    print(inp)
##    print(tar)
##    print ('11111111')
##    print ('-'*20)
#
#train_example = db_train.map(dataset.tf_encode)
#test_example = db_test.map(dataset.tf_encode)
dataset = Dataset("/home/wrench/下载/ML_Tech_Stack/Natural_language_process/data_onvif/2/brand")
train_examples, val_examples =  dataset.format_train_test()
dataset.build_train_test(test=test_partition)
for (step_idx, (inp)) in enumerate(train_examples):
    print(inp)
    print ('-'*20)

tf.Tensor(b'onvif www onvif org type video encoder onvif www onvif org profile streaming onvif www onvif org profile g onvif www onvif org type audio encoder onvif www onvif org hardware ds 2cd2710xyz abcd onvif www onvif org name hikvision 20ds 2cd2710xyz abcd onvif www onvif org location city hangzhou\t1', shape=(), dtype=string)
--------------------
tf.Tensor(b'onvif www onvif org type video encoder onvif www onvif org profile streaming onvif www onvif org profile g onvif www onvif org type audio encoder onvif www onvif org hardware ds 2cd2710xyz abcd onvif www onvif org name hikvision 20ds 2cd2710xyz abcd onvif www onvif org location city hangzhou\t1', shape=(), dtype=string)
--------------------
tf.Tensor(b'onvif www onvif org type video encoder onvif www onvif org profile streaming onvif www onvif org profile g onvif www onvif org type audio encoder onvif www onvif org hardware ds 2cd2710xyz abcd onvif www onvif org name hikvision 20ds 2cd2710xyz abcd onvif www onvif org location

In [6]:
tokenizer_source, tokenizer_target = dataset.tokenizer(train_examples)

2020-03-01 04:53:27,428 - absl - INFO - SubwordTextEncoder build: trying min_token_count 304
2020-03-01 04:53:27,452 - absl - INFO - SubwordTextEncoder build: trying min_token_count 157
2020-03-01 04:53:27,468 - absl - INFO - SubwordTextEncoder build: trying min_token_count 83
2020-03-01 04:53:27,487 - absl - INFO - SubwordTextEncoder build: trying min_token_count 46
2020-03-01 04:53:27,509 - absl - INFO - SubwordTextEncoder build: trying min_token_count 28
2020-03-01 04:53:27,522 - absl - INFO - SubwordTextEncoder build: trying min_token_count 19
2020-03-01 04:53:27,544 - absl - INFO - SubwordTextEncoder build: trying min_token_count 14
2020-03-01 04:53:27,564 - absl - INFO - SubwordTextEncoder build: trying min_token_count 12
2020-03-01 04:53:27,579 - absl - INFO - SubwordTextEncoder build: trying min_token_count 11


# 构建训练集和测试集

In [7]:
#MAX_LENGTH = 2100
#BATCH_SIZE = 32
#BUFFER_SIZE = 15000


# 训练集
#train_dataset = (train_example  # 输出：(英文句子，标签)
#                 .filter(dataset.filter_max_length)
#                 .shuffle(BUFFER_SIZE) # 将例子随机打乱
#                 .padded_batch(BATCH_SIZE, # 将 batch 里的序列都 pad 到同样长度
#                              padded_shapes=([MAX_LENGTH], []))
#                 .prefetch(tf.data.experimental.AUTOTUNE)) # 加速
## 验证集
#val_dataset = (test_example
#               .filter(dataset.filter_max_length)
#               .shuffle(BUFFER_SIZE)
#               .padded_batch(BATCH_SIZE,
#                             padded_shapes=([MAX_LENGTH], [])))
train_dataset = train_examples.map(dataset.tf_encode)
train_dataset = train_dataset.filter(dataset.filter_max_length)
train_dataset = train_dataset.cache()
train_dataset = train_dataset.shuffle(BUFFER_SIZE).padded_batch(
    BATCH_SIZE, padded_shapes=([MAX_LENGTH], [])
)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

val_dataset = val_examples.map(dataset.tf_encode)
val_dataset = val_dataset.filter(dataset.filter_max_length).padded_batch(
    BATCH_SIZE, padded_shapes=([MAX_LENGTH], [])
)

for (step_idx, (inp, tar)) in enumerate(train_dataset):
    print(inp)
    print(tar)
    print ('-'*20)
#    x_point, y_point = tf.convert_to_tensor([inp]), tf.convert_to_tensor([tar])
#    print(x_point.get_shape())
#    print(y_point.get_shape())
#    print ('222222222')

tf.Tensor(
[[288  29 289   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [288   2   7   4   1   3   8   1   3 135   1   2  16   4   1  11  14   6
    5   1   9  12  17   6   5   1  10  15  13 289   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0]], shape=(2, 50), dtype=int64)
tf.Tensor([0 0], shape=(2,), dtype=int64)
--------------------
tf.Tensor(
[[288   2   7   4   1   3   8   1   3 135   1   2  16   4   1  11  14   6
    5   1   9  12  17   6   5   1  10  15  13 289   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [288  10  23  24   1   9  25   1  11  30  31   1   3   8   1   2  20   7
   19   1  22  18  21 289   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0]], shape=(2, 50), dtype=int64)
tf.Tensor([0 1], shape=(2,)

# 4.2. Masking
遮掩（masking）可以说是在实践 Transformer 时最重要却也最容易被搞砸的一环。
它让 Transformer 在进行自注意力机制（Self-Attention Mechanism）时不至于偷看到不该看的内容。
在 Transformer 里有两种 masks：
1. padding mask
2. look ahead mask

Padding mask 是让 Transformer 用来识别序列实际的内容到哪里。此遮掩负责的就是将序列中被补 0 的地方（也就是 &#60;pad&#62;）的位置盖住，让 Transformer 可以避免「关注」到这些位置。

Look ahead mask 是用来确保 Decoder 在进行自注意力机制时每个子词只会「往前看」：关注（包含）自己之前的子词，不会不小心关注「未來」Decoder 产生的子词。

不管是哪一种 Masking 向量，那些值为 1 的位置就是遮掩存在的地方。

In [8]:
#def create_padding_mask(seq):
#    # padding mask 的工作就是把索引序列中為 0 的位置設為 1
#    mask = tf.cast(tf.equal(seq, 0), tf.float32)
#    return mask[:, tf.newaxis, tf.newaxis, :] #　broadcasting

def create_padding_mask(seq):
    """
    Mask all the pad tokens in the batch of sequence. It ensures that the model
    does not treat padding as the input. The mask indicates where pad value 0
    is present: it outputs a 1 at those locations, and a 0 otherwise.
    """
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)

    # add extra dimensions so that we can add the padding
    # to the attention logits.
    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)


def create_look_ahead_mask(size):
    """
    The look-ahead mask is used to mask the future tokens in a sequence. In other words,
    the mask indicates which entries should not be used.
    This means that to predict the third word, only the first and second word will be used.
    Similarly to predict the fourth word, only the first, second and the third word will
    be used and so on.
    """
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask  # (seq_len, seq_len)


def create_masks(inp, tar):
    # Encoder padding mask
    enc_padding_mask = create_padding_mask(inp)

    # Used in the 2nd attention block in the decoder.
    # This padding mask is used to mask the encoder outputs.
    # dec_padding_mask = create_padding_mask(inp)

    # Used in the 1st attention block in the decoder.
    # It is used to pad and mask future tokens in the input received by
    # the decoder.
    # look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[0])
    # dec_target_padding_mask = create_padding_mask(tar)
    # combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

    return enc_padding_mask


### 4.2.1 positional_encoding

In [9]:
def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return pos * angle_rates


def positional_encoding(position, d_model):
    """
    Since this model doesn't contain any recurrence or convolution, positional
    encoding is added to give the model some information about the relative
    position of the words in the sentence.
    The positional encoding vector is added to the embedding vector. Embeddings
    represent a token in a d-dimensional space where tokens with similar meaning
    will be closer to each other. But the embeddings do not encode the relative
    position of words in a sentence. So after adding the positional encoding,
    words will be closer to each other based on the similarity of their meaning
    and their position in the sentence, in the d-dimensional space
    """

    angle_rads = get_angles(
        np.arange(position)[:, np.newaxis], np.arange(d_model)[np.newaxis, :], d_model
    )

    # apply sin to even indices in the array; 2i
    sines = np.sin(angle_rads[:, 0::2])

    # apply cos to odd indices in the array; 2i+1
    cosines = np.cos(angle_rads[:, 1::2])

    pos_encoding = np.concatenate([sines, cosines], axis=-1)

    pos_encoding = pos_encoding[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)

### 4.2.2 loss_Function

In [10]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=40000):
        """
            Use the Adam optimizer with a custom learning
            rate scheduler according to the formula in
            the paper.
        """
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)


loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


def loss_function(real,pred):
    """
    Since the target sequences are padded,
    it is important to apply a padding mask when
    calculating the loss.
    """
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

# 这里是文本分类，只用到编码器，所以不需要mask

## Attention

首先，每个 $query_i$ / $key_i$ / $value_i$ 向量是由输入 $x_i$ 乘以矩阵 $M_q$ / $M_k$ / $M_v$ 得到的。

注意力机制 Attention 就是拿一个查询向量（query）去跟一组键值对（key-value）做运算，最后产生一个输出。

我们会把这些 query 向量拼成 Q 矩阵，key 向量拼成 K 矩阵，value 向量拼成 V 矩阵，然后利用矩阵运算运算同时让多个查询跟一组 key-value 做运算，从而最大化计算效率。

而不管是查询 query、键 key、值 value 还是输出，全部都是向量。输出 $output_i$ 是各个 value 的加权平均，权重向量即为 $softmax(\frac{Qk_i^{T}}{\sqrt{d_k}})$ ，即是对应输入 $x_i$ 的查询向量 $query_i$ 与每个键 $key_j$ 计算匹配程度。匹配程度越高，则 $value_i$ 被提取的比例就越大。（[论文原文](https://arxiv.org/abs/1706.03762)称此匹配度计算的函数为 compatibility function。）公式中的 $d_k$ 是指键向量 k 的维度。

$$ Attention(Q,K,V) = softmax(\frac{QK^{T}}{\sqrt{d_k}}) V $$

softmax 函数作用于 logits score 向量 $y$ 的每一个分量 $y_i$，对应获得其 probabilities。

$$ softmax(y_i) = \frac{e^{y_i}}{\sum_j e^{y_j}} $$

In [11]:
def scaled_dot_product_attention(q, k, v, mask):
    """Calculate the attention weights.
    q, k, v must have matching leading dimensions.
    The mask has different shapes depending on its type(padding or look ahead)
    but it must be broadcastable for addition.
    Args:
        q: query shape == (..., seq_len_q, depth)
        k: key shape == (..., seq_len_k, depth)
        v: value shape == (..., seq_len_v, depth)
        mask: Float tensor with shape broadcastable
            to (..., seq_len_q, seq_len_k). Defaults to None.
    Returns:
        output, attention_weights
    """

    # (..., seq_len_q, seq_len_k)
    matmul_qk = tf.matmul(q, k, transpose_b=True)

    # scale matmul_qk
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # add the mask to the scaled tensor.
    if mask is not None:
        scaled_attention_logits += mask * -1e9

    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    attention_weights = tf.nn.softmax(
        scaled_attention_logits, axis=-1
    )  # (..., seq_len_q, seq_len_k)

    output = tf.matmul(attention_weights, v)  # (..., seq_len_v, depth)

    return output, attention_weights

## 4.3. Multi-Head Attention

**每个 Head 各司其职：你看你的，我看我的。**

将 Q、K、V 这三个张量先分别转换到 d_model 维空间，再将其拆成多个比较低维的 depth 维张量 N 次，然后将这些产生的小 q、小 k 以及小 v 分別扔进注意力函数得到 N 个结果。接著将这 N 个 head 的结果串接起來，最后通过一个线性转换就能得到 multi-head attention 的输出。

为了实现 multi-head attention，得先能把一个 head 拆成多个 heads。而这实际上就是把一个 d_model 维的向量拆成 num_heads 个 depth 维的向量，使得：num_heads * depth = d_model。

In [12]:
def split_heads_bak(x, d_model, num_heads):
    # x.shape: (batch_size, seq_len, d_model)
    batch_size = tf.shape(x)[0]

    # 确保维度 `d_model` 可以被平分成 `num_heads` 个 `depth` 维度，即整除
    assert d_model % num_heads == 0
    depth = d_model // num_heads  # 这是分成多头以后每个向量的维度

    # 将最后一个 d_model 维度分成 num_heads 个 depth 维度
    # 最后一个维度变成两个维度，张量 x 从 3 维到 4 维
    # (batch_size, seq_len, num_heads, depth)
    reshaped_x = tf.reshape(x, shape=(batch_size, -1, num_heads, depth))

    # 将 head 的维度拉伸使得最后两个维度为子词以及其对应的 depth 向量
    # (batch_size, num_heads, seq_len, depth)
    output = tf.transpose(reshaped_x, perm=[0, 2, 1, 3])

    return output

**Multi-Head Attention**

In [13]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()

        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)

        # (batch_size, num_heads, seq_len_q, depth)
        q = self.split_heads(q, batch_size)
        # (batch_size, num_heads, seq_len_k, depth)
        k = self.split_heads(k, batch_size)
        # (batch_size, num_heads, seq_len_v, depth)
        v = self.split_heads(v, batch_size)

        # scaled_attention.shape == (batch_size, num_heads, seq_len_v, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask
        )

        scaled_attention = tf.transpose(
            scaled_attention, perm=[0, 2, 1, 3]
        )  # (batch_size, seq_len_v, num_heads, depth)

        concat_attention = tf.reshape(
            scaled_attention, (batch_size, -1, self.d_model)
        )  # (batch_size, seq_len_v, d_model)

        # (batch_size, seq_len_v, d_model)
        output = self.dense(concat_attention)

        return output, attention_weights

## 4.4. Transformer

- Transformer
    - Encoder
        - 输入 Embedding
        - 位置 Encoding
        - N 个 Encoder layers
        - sub-layer 1: Encoder 自注意力机制
        - sub-layer 2: Feed Forward
    - Final Dense Layer

### 4.4.1. Position-wise Feed-Forward Networks

建立 Transformer 里 Encoder / Decoder layer 都会使用到的 Feed Forward 组件。

此函数在每次被 call 的时候都会回传一组新的全连接前馈神经网络（Fully-connected Feed Forward Network，FFN），其输入张量与输出张量的最后一个维度皆为 d_model，而在 FFN 中间层的维度則為 dff。一般会让 dff 大于 d_model，让 FFN 从输入的 d_model 维度里提取些有用的信息。在论文中 d_model 为 512，dff 则为 4 倍的 2048。两个都是可以调整的超参数。

In [14]:
def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential(
        [
            # (batch_size, seq_len, dff)
            tf.keras.layers.Dense(dff, activation="relu"),
            tf.keras.layers.Dense(d_model),  # (batch_size, seq_len, d_model)
        ]
    )


### 4.4.2. Encoder layer

有了 Multi-Head Attention（MHA）以及 Feed-Forward Network（FFN），就可以完成第一个 Encoder layer 了。

一个 Encoder layer 里会有两个 sub-layers，分別为 MHA 以及 FFN。在 Add & Norm 步骤里，每个 sub-layer 会有一个残差廉洁（residual connection）来帮助减缓梯度消失（Gradient Vanishing）的问题。接着两个 sub-layers 都会针对最后一维 d_model 做 layer normalization，将 batch 里每个子词的输出彼此独立地做转换，使其均值和标准差分別靠近 0 和 1 之后输出。

另外在将 sub-layer 的输出与其输入相加之前，还会做 regularization，对该 sub-layer 的输出使用 dropout。
(PS:这里不需要mask来进行遮掩)

In [15]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, x, training, mask):

        attn_output, attention_weights = self.mha(
            x, x, x, mask
        )  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training=training)
        # (batch_size, input_seq_len, d_model)
        out1 = self.layernorm1(x + attn_output)

        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(
            out1 + ffn_output
        )  # (batch_size, input_seq_len, d_model)

        return out2, attention_weights

### 4.4.5. Encoder

Encoder 里主要包含了以下 3 个组件：

1. 输入的词嵌入层
2. 位置编码
3. N 个 Encoder layers

由于大部分的工作都在 Encoder layer 里执行了，因此 Encoder 部分代码比较简单

In [16]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, rate=0.1):
        super(Encoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
        self.pos_encoding = positional_encoding(input_vocab_size, self.d_model)

        self.enc_layers = [
            EncoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)
        ]

        self.dropout = tf.keras.layers.Dropout(rate)

    def call(self, x, training, mask):

        seq_len = tf.shape(x)[1]

        # adding embedding and position encoding.
        x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        attention_weights = {}

        for i in range(self.num_layers):
            x, block1 = self.enc_layers[i](x, training, mask)

        attention_weights["decoder_layer{}_block1".format(i + 1)] = block1

        return x, attention_weights  # (batch_size, input_seq_len, d_model)

### 4.4.6  Modeler

In [17]:
class Transformer(tf.keras.Model):
    def __init__(
        self,
        num_layers,
        d_model,
        num_heads,
        dff,
        input_vocab_size,
        target_vocab_size,
        rate=0.1,
    ):
        super(Transformer, self).__init__()

        self.encoder = Encoder(
            num_layers, d_model, num_heads, dff, input_vocab_size, rate
        )

        self.flatten = tf.keras.layers.Flatten()
        self.final_layer = tf.keras.layers.Dense(
            64)
        self.dense_1 = tf.keras.layers.Dense(
            target_vocab_size, activation="softmax")
        # self.out = tf.nn.softmax(target_vocab_size)
        #        self.dense_1 = tf.keras.layers.Dense(target_vocab_size, activation="softmax")
    def call(
        self, inp, tar, training, enc_padding_mask, look_ahead_mask, dec_padding_mask
    ):

        enc_output, attention_weights = self.encoder(
            inp, training, enc_padding_mask
        )  # (batch_size, inp_seq_len, d_model)

        predictions = self.final_layer(
            enc_output
        )  # (batch_size, tar_seq_len, target_vocab_size)

        embeddings = self.flatten(predictions)
        predictions = self.dense_1(embeddings)

        return predictions, embeddings, attention_weights

In [18]:
train_perc = 100
num_layers = 1 
d_model = 12
dff = 32 
num_heads = 1
input_vocab_size = tokenizer_source.vocab_size + 2
target_vocab_size = tokenizer_target.num_classes
dropout_rate =0.5

In [19]:
transformer = Transformer(num_layers, d_model, num_heads, dff, 
                          input_vocab_size, target_vocab_size, dropout_rate)

print(f"""这个 Transformer 有 {num_layers} 层 Encoder / Decoder layers
d_model: {d_model}
num_heads: {num_heads}
dff: {dff}
input_vocab_size: {input_vocab_size}
target_vocab_size: {target_vocab_size}
dropout_rate: {dropout_rate}
""")

这个 Transformer 有 1 层 Encoder / Decoder layers
d_model: 12
num_heads: 1
dff: 32
input_vocab_size: 290
target_vocab_size: 2
dropout_rate: 0.5



In [20]:
train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
        name="train_accuracy")
test_loss = tf.keras.metrics.Mean(name="test_loss")
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
        name="test_accuracy")

@tf.function # 利用 TensorFlow 将 eager code 优化并加快运算
def train_step(inp, tar):
    # 建立 3 个遮掩
    logger.debug("input: {}".format(inp.shape))
    logger.debug("target: {}".format(tar.shape))
    
    enc_padding_mask = create_masks(inp, tar)

    # 记录 Transformer 的所有运算过程以方便之后做梯度下降
    with tf.GradientTape() as tape:
        # 注意是输入 `tar_inp` 而非 `tar`，记得将 `training` 参数设定为 True
        predictions, enc_output ,_ = transformer(inp, tar, 
                                     True, 
                                     enc_padding_mask, 
                                     None, 
                                     None)
        # 跟影片中显示的相同，计算左移一個字的序列跟模型预测分布之间的差异，当作 loss
        loss = loss_function(tar, predictions)

    # 取出梯度并呼叫前面定义的 Adam optimizer 以便更新 Transformer 里可训练的参数
    gradients = tape.gradient(loss, transformer.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

    # （非必要）将 loss 以及训练 acc 记录到 TensorBoard 上
    train_loss(loss)
    train_accuracy(tar, predictions)
    
    logger.debug("predictions: {}".format(predictions.shape))
    logger.debug("tar_real: {}".format(tar.shape))
    logger.debug("enc_output: {}".format(enc_output.shape))

##  4.7. Optimizer

这里跟[原论文](https://arxiv.org/abs/1706.03762)保持一致，使用 Adam Optimizer 以及自定义的 learning rate scheduler

$$ learning\_rate = d_{model}^5 \times \min{step\_num^{-0.5}, step\_num \times warmup\_steps^{-1.5}} $$

这个 schedule 让训练过程的前 warmup_steps 的 learning rate 线性增加，之后则跟步骤数 step_num 的反平方根成比例下降。

可以参考[ TensorFlow 官方教学的实现](https://www.tensorflow.org/beta/tutorials/text/transformer?authuser=1#optimizer)

In [21]:
# 将自定义的 learning rate schdeule 输入 Adam opt.
# Adam opt. 的参数都跟原论文相同
learning_rate = CustomSchedule(d_model)
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)

## 5.1. checkpoint

设置 checkpoint 来定期存储 / 读取模型及 optimizer 是必备的。

本节会定义一个 checkpoint 路径，此路径包含了各种超参数的信息，方便之后比较不同试验的结果并载入已训练的进度。

同时也需要一个 checkpoint manager 来做所有跟存储模型模有关的杂事，并只保留最新 5 个 checkpoints 以避免占用太多空间。

In [22]:
import os
output_dir = "/home/wrench/data/NMT"
checkpoint_path = os.path.join(output_dir, "checkpoints")
log_dir = os.path.join(output_dir, 'logs')
# 方便比较不同实验 / 不同超参数设定的结果
run_id = f"{num_layers}layers_{d_model}d_{num_heads}heads_{dff}dff_{train_perc}train_perc"
checkpoint_path = os.path.join(checkpoint_path, run_id)
log_dir = os.path.join(log_dir, run_id)

# tf.train.Checkpoint 可以帮我们把想要存下來的东西整合起來，方便存储与读取
# 一般來说会存下模型以及 optimizer 的状态
ckpt = tf.train.Checkpoint(transformer=transformer, 
                           optimizer=optimizer)

# ckpt_manager 会去 checkpoint_path 看有没有符合 ckpt 里定义的东西
# 存档的时候只保留最近 5 次 checkpoints，其他自动删除
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# 如果在 checkpoint 路径上有发现档案就读进来
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)

    # 用來确认之前训练多少 epochs 了
    last_epoch = int(ckpt_manager.latest_checkpoint.split("-")[-1])
    print(f'已读取最新的 checkpoint，模型已训练 {last_epoch} epochs。')
else:
    last_epoch = 0
    print("没找到 checkpoint，从头训练。")

没找到 checkpoint，从头训练。


In [23]:
from sklearn.metrics import f1_score
def test_acc(batch=32, test_dataset=[], transformer=[], test_accuracy=[], test_loss=[]):
    real = []
    pred = []

    for (batch, (inp, tar)) in enumerate(test_dataset):
        #logger.debug("input: {}".format(inp.shape))
        #logger.debug("target: {}".format(tar.shape))

        enc_padding_mask = create_masks(inp, tar)

        predictions, _, _ = transformer(
            inp, tar, False, enc_padding_mask, None, None)
        #logger.debug("predictions: {}".format(predictions.shape))
        #logger.debug("tar_real: {}".format(tar.shape))

        test_accuracy(tar, predictions)
        test_loss(loss_function(tar, predictions))

        tar = tf.Variable(tar)
        predictions = tf.Variable(predictions)

        real += tar.numpy().tolist()
        pred += [i for i in np.argmax(predictions.numpy(), axis=1)]

        test_accuracy(tar, predictions)
        test_loss(loss_function(tar, predictions))

    # print(classification_report(real, pred))
    F1 = f1_score(real, pred, average='macro')

    return F1


# 正式训练

In [24]:
import time
EPOCHS = 50

print(f"此超参数組合的 Transformer 已经训练 {last_epoch} epochs。")
print(f"剩余 epochs：{min(0, last_epoch - EPOCHS)}")

# 用来输入信息到 TensorBoard（非必要但十分推荐）
summary_writer = tf.summary.create_file_writer(log_dir)
# training loop
best_test_acc = 0
best_test_f1 = 0
# 比对设定的 `EPOCHS` 以及已训练的 `last_epoch` 来决定还要训练多少 epochs
for epoch in range(last_epoch, EPOCHS):
    start = time.time()

    # 重置记录 TensorBoard 的 metrics
    train_loss.reset_states()
    train_accuracy.reset_states()
   
    # 一个 epoch 就是把定义的训练数据集一个一个 batch 地拿出来处理，直到看完整个数据集
    for (step_idx, (inp, tar)) in enumerate(train_dataset):
        train_step(inp=inp,tar=tar)

    # 每个 epoch 完成就存一次档    
#    if (epoch + 1) % 1 == 0:
#        ckpt_save_path = ckpt_manager.save()
#        print ('Saving checkpoint for epoch {} at {}'.format(epoch+1, ckpt_save_path))
    test_accuracy.reset_states()
    test_loss.reset_states()
    test_f1 = test_acc(
        batch=2,
        test_dataset=val_dataset,
        transformer=transformer,
        test_accuracy=test_accuracy,
        test_loss=test_loss,
        )    
    print(
            "Epoch {} Test Loss {:.4f} Accuracy {:.4f} F1 {:.4f}".format(
                epoch + 1, test_loss.result(), test_accuracy.result(), test_f1
            )
        )
    if best_test_f1 < test_accuracy.result():
        ckpt_save_path = ckpt_manager.save()
        print(
            "Saving checkpoint for test epoch {} at {}".format(
                epoch + 1, ckpt_save_path)
            )
        best_test_f1 = test_accuracy.result()
        print("test Time taken for 1 epoch: {} secs\n".format(time.time() - start))        

    # 将 loss 以及 accuracy 写到 TensorBoard 上
    with summary_writer.as_default():
        tf.summary.scalar("train_loss", train_loss.result(), step=epoch + 1)
        tf.summary.scalar("train_acc", train_accuracy.result(), step=epoch + 1)

    print('Epoch {} Train Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1, 
                                                        train_loss.result(), 
                                                        train_accuracy.result()))
    print('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

此超参数組合的 Transformer 已经训练 0 epochs。
剩余 epochs：-50
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7fc157822df0> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2020-03-01 04:53:28,936 - root - DEBUG - input: (2, 50)2020-03-01 04:53:30,306 - tensorflow - WARNING - AutoGraph could not transform <bound method Socket.send of <zmq.sugar.socket.Socket object at 0x7fc15438ce80>> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7fc157822df0> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7fc157822df0> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert



2020-03-01 04:53:30,771 - root - DEBUG - target: (2,)
2020-03-01 04:53:31,611 - root - DEBUG - predictions: (2, 2)
2020-03-01 04:53:31,623 - root - DEBUG - tar_real: (2,)
2020-03-01 04:53:31,635 - root - DEBUG - enc_output: (2, 3200)
2020-03-01 04:53:31,664 - root - DEBUG - input: (2, 50)
2020-03-01 04:53:31,676 - root - DEBUG - target: (2,)
2020-03-01 04:53:31,885 - root - DEBUG - predictions: (2, 2)
2020-03-01 04:53:31,897 - root - DEBUG - tar_real: (2,)
2020-03-01 04:53:31,910 - root - DEBUG - enc_output: (2, 3200)
2020-03-01 04:53:32,108 - root - DEBUG - input: (1, 50)
2020-03-01 04:53:32,119 - root - DEBUG - target: (1,)
2020-03-01 04:53:32,325 - root - DEBUG - predictions: (1, 2)
2020-03-01 04:53:32,336 - root - DEBUG - tar_real: (1,)
2020-03-01 04:53:32,349 - root - DEBUG - enc_output: (1, 3200)
/home/wrench/miniconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predict

Epoch 1 Test Loss 0.1455 Accuracy 0.3125 F1 0.2381
Saving checkpoint for test epoch 1 at /home/wrench/data/NMT/checkpoints/1layers_12d_1heads_32dff_100train_perc/ckpt-1
test Time taken for 1 epoch: 4.378221273422241 secs

Epoch 1 Train Loss 0.2401 Accuracy 0.4172
Time taken for 1 epoch: 4.379403829574585 secs

Epoch 2 Test Loss 0.1445 Accuracy 0.3125 F1 0.2381
Epoch 2 Train Loss 0.2617 Accuracy 0.3907
Time taken for 1 epoch: 0.1533496379852295 secs

Epoch 3 Test Loss 0.1430 Accuracy 0.3125 F1 0.2381
Epoch 3 Train Loss 0.2501 Accuracy 0.4371
Time taken for 1 epoch: 0.15086627006530762 secs

Epoch 4 Test Loss 0.1421 Accuracy 0.3125 F1 0.2381
Epoch 4 Train Loss 0.2540 Accuracy 0.4106
Time taken for 1 epoch: 0.14279890060424805 secs

Epoch 5 Test Loss 0.1381 Accuracy 0.3125 F1 0.2381
Epoch 5 Train Loss 0.2415 Accuracy 0.4106
Time taken for 1 epoch: 0.14506244659423828 secs

Epoch 6 Test Loss 0.1358 Accuracy 0.3125 F1 0.2381
Epoch 6 Train Loss 0.2674 Accuracy 0.3974
Time taken for 1 epoch: 

## 预测数据

In [25]:
tf.keras.backend.clear_session()

def evaluate(inp_sentence,tokenizer_source, tokenizer_target, transformer):
    
    inp = [tokenizer_source.encode(inp_sentence)]
    inp = tf.keras.preprocessing.sequence.pad_sequences(
        inp, maxlen=50, padding="post"
    )
    print(inp)
    # inp = tf.expand_dims(inp, 0)
    enc_padding_mask = create_masks(inp, None)

    # predictions.shape == (batch_size, seq_len, vocab_size)
    predictions, _, attention_weights = transformer(
        inp, None, False, enc_padding_mask, None, None
    )

    predictions = tf.squeeze(predictions, axis=0)
    print(predictions)
    predictions_index = tf.cast(
        tf.argsort(predictions, axis=-1, direction="DESCENDING"), tf.int32
    )
    print(predictions_index)
    predictions = predictions.numpy()[predictions_index.numpy()]
    print(predictions)
    _pred = [
        {"score": float(i), "label": tokenizer_target.int2str(j.numpy())}
        for i, j in zip(predictions, predictions_index)
    ][:2]

    return _pred, attention_weights


def translate(sentence,tokenizer_source, tokenizer_target, transformer):
    predictions, attention = evaluate(
        sentence,tokenizer_source, tokenizer_target, transformer
    )

    return predictions, attention


predictions, attention = translate("dahua",tokenizer_source,tokenizer_target,transformer)

print(predictions)

predictions, attention = translate("hikvision",tokenizer_source,tokenizer_target,transformer)

print(predictions)


[[25  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0]]
tf.Tensor([7.2733146e-06 9.9999273e-01], shape=(2,), dtype=float32)
tf.Tensor([1 0], shape=(2,), dtype=int32)
[9.9999273e-01 7.2733146e-06]
[{'score': 0.9999927282333374, 'label': '0'}, {'score': 7.273314622580074e-06, 'label': '1'}]
[[29  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0]]
tf.Tensor([0.00221491 0.9977851 ], shape=(2,), dtype=float32)
tf.Tensor([1 0], shape=(2,), dtype=int32)
[0.9977851  0.00221491]
[{'score': 0.9977850914001465, 'label': '0'}, {'score': 0.0022149106953293085, 'label': '1'}]
